In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [3]:
df = pd.read_csv('Resources/LoanStats_2019Q1.csv')

In [5]:
# display(df.tail())
df = df[:-2]
# display(df.tail())

In [6]:
x_cols = [i for i in df.columns if i not in ('id', 'member_id', 'loan_status')]
X = df[x_cols]
y = df['loan_status']

In [7]:
X.tail()

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,...,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
115668,15000.0,15000.0,15000.0,60 months,13.56%,345.62,C,C1,Lead counselor,10+ years,...,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN
115669,40000.0,40000.0,40000.0,36 months,6.46%,1225.24,A,A1,President - North America,4 years,...,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN
115670,5000.0,5000.0,5000.0,36 months,13.56%,169.83,C,C1,NaN,NaN,...,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN
115671,6000.0,6000.0,6000.0,36 months,6.46%,183.79,A,A1,NaN,< 1 year,...,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN
115672,16000.0,16000.0,16000.0,36 months,16.14%,563.62,C,C4,Estimator/Supervisor,10+ years,...,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
y

0          Issued
1          Issued
2          Issued
3          Issued
4          Issued
           ...   
115668    Current
115669    Current
115670    Current
115671    Current
115672    Current
Name: loan_status, Length: 115673, dtype: object

In [9]:
# columns to encode: grade, sun_grade, term, loan_amnt


from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df2 = df.copy()
df2["grade"] = le.fit_transform(df["grade"])
df2.head() 

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
0,NaN,NaN,20000.0,20000.0,20000.0,60 months,17.19%,499.10,2,C5,...,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,21225.0,21225.0,21225.0,60 months,14.74%,502.05,2,C2,...,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,5000.0,5000.0,5000.0,36 months,17.97%,180.69,3,D1,...,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,20000.0,20000.0,20000.0,36 months,8.19%,628.49,0,A4,...,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,12000.0,12000.0,12000.0,60 months,15.57%,289.09,2,C3,...,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
grades = []
for i in X.orig_projected_additional_accrued_interest:
    if i in grades:
        pass
    else:
        grades.append(i)
        
# grades = grades[:-1]
grades = sorted(grades)
len(grades)

KeyboardInterrupt: 

In [25]:
X.orig_projected_additional_accrued_interest.dropna()

Series([], Name: orig_projected_additional_accrued_interest, dtype: float64)

In [ ]:
grade_num = {
    "A": 1,
    "B": 2,
    "C": 3,
    "D": 4,
    "E": 5,
    "F": 6,
    "G": 7
}

In [ ]:
X["grade_num"] = X["grade"].apply(lambda x: grade_num[x])
X.grade_num

In [ ]:
test = df[:-2]
test.tail()

In [ ]:
grades = []
for i in test.grade:
    if i in grades:
        pass
    else:
        grades.append(i)
        
#grades = grades[:-1]
grades = sorted(grades)
grades

In [ ]:
test["grade_num"] = test["grade"].apply(lambda x: grade_num[x])
test.grade_num